# Knjižnica Polars in primerjava s Pandas

Do zdaj ste že dobro spoznali knjižnico Pandas, ki je najpogosteje uporabljena knjižnica za delo s podatki, v tej enoti pa boste spoznali še knjižnico Polars. Knjižnica Polars je alternativa knjižnici Pandas ponuja pa nekaj izboljšav, ki se predvsem odražajo v hitrosti delovanja funkcionalnosti knjižnice. Gre torej za pohitritve nalaganja, shranjevanja in obdelave podatkov, kar je vedno dobrodošlo pri obdelavi velepodatkov. V nadaljevanju bomo na primerih primerjali delo s Polars in Pandas in ugotavljali posebnosti knjižnice Polars.

## Namestitev Polars

Knjižnico Polars namestimo kot vsako drugo knjižnico za programski jezik Python. Za namestitev uporabimo orodje ``pip``, pri uporabi pa ponavadi uporabimo alias ``pl`` (podobno kot pri Pandas uporabimo alias ``pd``).

In [ ]:
%pip install polars --quiet

In [2]:
import pandas as pd
import polars as pl # alias knjižnice Polars

## Delo s Polars in primerjava s Pandas

Ena najboljših lastnosti knjižnice Polars je to, da so imena funkcij v veliki večini enaka kot v knjižnici Pandas. V praksi to pomeni, da lahko delo s knjižnico Polars na obstoječi programski kodi za Pandas zelo enostavno migriramo. To naredimo tako, da v naši programski kodi samo zamenjamo alias. Poglejmo si primer nalaganja manjše datoteke CSV v Pandas in Polars.

In [14]:
import numpy as np

def generate_data(size):
    df = pd.DataFrame()
    df['Polozaj'] = np.random.choice(['levo', 'sredina', 'desno'], size)
    df['Starost'] = np.random.randint(18, 50, size)
    df['Ekipa'] = np.random.choice(['rdeča', 'modra', 'rumena', 'zelena'], size)
    df['Zmaga'] = np.random.choice(['da', 'ne'], size)
    df['Verjetnost'] = np.random.uniform(0, 1, size)
    return df

def optimize_types(df):
    df['Polozaj'] = df['Polozaj'].astype('category')
    df['Ekipa'] = df['Ekipa'].astype('category')
    df['Starost'] = df['Starost'].astype('uint8')
    df['Verjetnost'] = df['Verjetnost'].astype('float32')
    df['Zmaga'].map({'da': True, 'ne': False})
    return df

df = optimize_types(generate_data(10_000_000))
df.to_csv('dataset_csv.csv', index=False)

In [3]:
# nalaganje podatkov s Pandas
%time df_pandas = pd.read_csv('dataset_csv.csv') # vrne Pandas DataFrame

df_pandas.info()
df_pandas.head()

CPU times: total: 1 s
Wall time: 2.59 s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Polozaj     object 
 1   Starost     int64  
 2   Ekipa       object 
 3   Zmaga       object 
 4   Verjetnost  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 381.5+ MB


,Polozaj,Starost,Ekipa,Zmaga,Verjetnost
0,sredina,41,modra,ne,0.681433
1,desno,35,rdeča,ne,0.938699
2,levo,23,modra,ne,0.445838
3,levo,46,zelena,ne,0.819481
4,levo,44,rumena,da,0.170575


In [11]:
# nalaganje podatkov s Polars
%time df_polars = pl.read_csv('dataset_csv.csv') # vrne Polars DataFrame

df_polars.head()

CPU times: total: 1.98 s
Wall time: 155 ms


Polozaj,Starost,Ekipa,Zmaga,Verjetnost
str,i64,str,str,f64
"""sredina""",41,"""modra""","""ne""",0.6814334
"""desno""",35,"""rdeča""","""ne""",0.938699
"""levo""",23,"""modra""","""ne""",0.445838
"""levo""",46,"""zelena""","""ne""",0.8194815
"""levo""",44,"""rumena""","""da""",0.1705755


Vidimo, da smo s knjižnico Pandas naložili datoteko CSV v okoli 2 sekundah, s knjižnico Polars pa v približno 100 milisekundah. To je velika pohitritev, ki izhaja iz zalednih struktur obeh knjižnic. Knjižnica Pandas v ozadju uporablja strukture knjižnice numpy, dočimer knjižnica Polars v ozadju uporablja strukture knjižnice Apache Arrow. Knjižnica Polars je tako boljša za nalaganje večje količine velepodatkov.

Druga stvar, ki jo lahko opazimo je, da smo za nalaganje uporabili praktično enako kodo - zamenjali smo le alias iz ``pd`` v ``pl``. Pri tem je vredno omeniti, da Polars vrača podatkovne tipe že v izpisu funkcije ``.head()`` in tako nima funkcije, ki bi bila podobna funkciji ``.info()`` v Pandas.

Poglejmo kako izgleda filtriranje oziroma poizvedovanje po podatkovni zbirki v obeh knjižnicah. Pri tem bomo uporabili ustvarjeno podatkovno zbirko iz prejšnje enote, ki vsebuje 10 milijonov vrstic.

Primerjavo ukazov Pandas in Polars lahko pregledate v spodnjih virih:

* [Dokumentacija Polars](https://docs.pola.rs/)
* [Cheatsheet for Polars to Pandas](https://www.rhosignal.com/posts/polars-pandas-cheatsheet/)

In [5]:
# izbira podatkov in grupiranje s Pandas
%timeit df_pandas.query('`Starost` > 18 & `Starost` < 25').groupby('Polozaj').size() 

164 ms ± 2.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
# izbira podatkov in grupiranje s Polars
%timeit df_polars.filter((pl.col('Starost') > 18) & (pl.col('Starost') < 25)).group_by('Polozaj').len() 

25 ms ± 581 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%%timeit 

# veriženje v Pandas
(df_pandas
 .query('`Starost` > 18 & `Starost` < 25')
 .groupby('Polozaj')
 .size()
)

164 ms ± 987 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%timeit 

# veriženje v Polars
(df_polars
    .filter((pl.col('Starost') > 18) & (pl.col('Starost') < 25))
    .group_by('Polozaj')
    .len()
)

24.8 ms ± 400 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Opazimo, da so funkcije knjižnice Polars za ta tip operacij bistveno hitrejše! Hkrati vidimo, da je način uporabe Polars zelo podoben Pandas in da podpira tudi veriženje funkcij. Preverimo še učinkovitost urejanja podatkov.

In [9]:
# urejanje podatkov s Pandas
%timeit df_pandas.sort_values(by='Verjetnost', ascending=False).head(10)

2.09 s ± 9.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# urejanje podatkov s Polars
%timeit df_polars.sort('Verjetnost', descending=True).head(10)

374 ms ± 52.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Knjižnica Polars je tudi pri sortiranju bistveno hitrejša od knjižnice Pandas. Na podlagi rezultatov teh preizkusov lahko zaključimo, da je knjižnica Polars za velike količine podatkov boljša izbira kot knjižnica Pandas. Čeprav je Pandas veliko bolj razširjena knjižnica za obdelavo podatkov v podatkovni znanosti, je vredno razmisliti o izboljšavah, ki so na voljo. Polars ni edina alternativa za pohitritev dela z velepodatki. Velepodatke lahko obdelujemo tudi z grafičnimi procesorskimi enotami, kar je še posebej uporabno pri učenju modelov umetne inteligence. S tem pristopom se boste spoznali v naslednji enoti.

Za vajo lahko poskusite sami poskusiti naslednje poizvedbe nad ustvarjeno podatkovno zbirko. Pri tem poskusite s knjižnicama Pandas in Polars.

* Izberite vse igralce na levem položaju, ki igrajo za modro ekipo, jih uredite po letih padajoče in izberite prvih 15.
* Izberite vse igralce na levem ali desnem položaju, ki igrajo za rdečo ali rumeno ekipo in imajo verjetnost za zmago.
* Izračunajte povprečno verjetnost zmage za vse igralce po posameznih ekipah.